In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import torch.nn as nn
import torch
from dataset import ManualFeatureDataModule
from transformer import TransformerClassifierModule

In [2]:
data_dir = Path("/media/hdd1/i-care/6h-combined")
label_dir = Path("/home/bc299/icare/artifacts/patient_data.csv")
dm = ManualFeatureDataModule(root_dir=data_dir, labels_csv=label_dir)
dm.setup()

x, y = next(iter(dm.train_dataloader()))
x.size()

torch.Size([32, 296, 144])

In [5]:
model = TransformerClassifierModule(input_size=296,
                                    d_model=128,
                                    nhead=8,
                                    num_layers=2,
                                    dropout=0.2,
                                    learning_rate=1e-4)
model(x)

tensor([[-0.1097, -2.2645],
        [-0.1084, -2.2753],
        [-0.3543, -1.2095],
        [-0.1733, -1.8379],
        [-0.1645, -1.8859],
        [-0.1355, -2.0660],
        [-0.1811, -1.7978],
        [-0.2624, -1.4662],
        [-0.1853, -1.7768],
        [-0.2557, -1.4888],
        [-0.1952, -1.7298],
        [-0.3167, -1.3039],
        [-0.1828, -1.7894],
        [-0.2497, -1.5098],
        [-0.2744, -1.4273],
        [-0.3110, -1.3195],
        [-0.2081, -1.6722],
        [-0.1770, -1.8187],
        [-0.3365, -1.2527],
        [-0.2682, -1.4473],
        [-0.1194, -2.1845],
        [-0.3482, -1.2239],
        [-0.1663, -1.8760],
        [-0.2315, -1.5766],
        [-0.4737, -0.9746],
        [-0.2434, -1.5324],
        [-0.1459, -1.9970],
        [-0.1685, -1.8640],
        [-0.1827, -1.7898],
        [-0.2877, -1.3862],
        [-0.2199, -1.6226],
        [-0.1793, -1.8069]], grad_fn=<LogSoftmaxBackward0>)

In [72]:
d_model = 128
n_head = 8
num_layers = 2
dropout = 0.3

embedding = nn.Sequential(
    nn.Linear(14, d_model),
    nn.ReLU(),
)
conv = nn.Conv1d(d_model, d_model, kernel_size=5, padding=2)
pe = nn.Parameter(torch.randn(1, d_model))
transformer = nn.Transformer(d_model, n_head, num_layers, dropout=dropout)
fc = nn.Linear(d_model, 2)

output = conv(embedding(x).permute(0, 2, 1)).permute(0, 2, 1)
output = output + pe
output = output.permute(1, 0, 2)
out = transformer.encoder(output)
out = out[-1, :, :]
out = fc(out)
out.size()

torch.Size([32, 2])

In [19]:
from pathlib import Path
path = Path("/media/hdd1/i-care/combined-features/0284/0284_007_features.npy")
import numpy as np
np.load(path).shape

(296, 12)